In [3]:
import rospy
from hbp_nrp_virtual_coach.virtual_coach import VirtualCoach
import os, shutil

from ipywidgets import FloatProgress
from IPython.display import display
import time

In [4]:
vc = VirtualCoach(environment='local', storage_username='nrpuser')
# Type "password"

INFO: [2018-10-11 16:03:58,305 - Configuration] Loading configuration file config.json
INFO: [2018-10-11 16:03:58,308 - Configuration] Using user specified environment: local
INFO: [2018-10-11 16:03:58,313 - VirtualCoach] Logging into the Storage Server as: nrpuser
········
INFO: [2018-10-11 16:04:01,199 - VirtualCoach] Ready.


In [5]:
configName = "fable_cerebellum_hybrid"

In [67]:
folder = "/home/dtu-neurorobotics/Documents/workspace/Projects/DTU-Pavia_Cerebellum_Carlos_hybrid/cereb_tuning/results/"
test = "6"
folder = folder+test
try:
    file_list = os.listdir(folder)
except Exception as e:
    print("Creating experiment folder: " + test)
    os.mkdir(folder)
    file_list = os.listdir(folder)
    
    
        
if len(file_list):
    print("Going to delete everything here:")
    #print(file_list)
    answer = raw_input("You sure? (y/n)")

    if answer == "y":
        
        for the_file in os.listdir(folder):
            file_path = os.path.join(folder, the_file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path): 
                    shutil.rmtree(file_path)

            except Exception as e:
                print(e)

        print("Success")
    else:
        print("Cancelled")
else:
    print("Empty directory")

Going to delete everything here:
You sure? (y/n)y
Success


In [68]:
experiment = vc.clone_experiment_to_storage(configName)
print(experiment)

INFO: [2018-10-15 12:25:50,521 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-10-15 12:25:50,559 - VirtualCoach] Experiment "fable_cerebellum_hybrid" cloned successfully
fable_manipulation_dtu_spiking_cereb_hybrid_0


In [69]:
sim = vc.launch_experiment(experiment)

INFO: [2018-10-15 12:25:56,784 - VirtualCoach] Preparing to launch fable_manipulation_dtu_spiking_cereb_hybrid_0.
INFO: [2018-10-15 12:25:56,788 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-10-15 12:25:56,799 - VirtualCoach] Retrieving list of available servers.
[{u'gzweb': {u'assets': u'http://localhost:8080/assets',
             u'nrp-services': u'http://localhost:8080',
             u'videoStreaming': u'http://localhost:8080/webstream/',
             u'websocket': u'ws://localhost:8080/gzbridge'},
  u'id': u'localhost',
  u'rosbridge': {u'websocket': u'ws://localhost:8080/rosbridge'},
  u'serverJobLocation': u'local'}]
INFO: [2018-10-15 12:25:56,809 - Simulation] Attempting to launch fable_manipulation_dtu_spiking_cereb_hybrid_0 on localhost.
INFO: [2018-10-15 12:26:09,894 - Simulation (fable_manipulation_dtu_spiking_cereb_hybrid_0 - localhost #14)] Simulation Successfully Created.
INFO: [2018-10-15 12:26:09,899 - Simulation (fable_manipulation_dtu_spiking_cereb_hybri

In [70]:
run_time = 200

sim.start()

walltime_start = time.time()

time_started = rospy.Time.now()
delta_t = rospy.Time.now() - time_started

print("\nStart delta_t: " + str(delta_t.to_sec()) + " (Should be 0)")

f = FloatProgress(min=0, max=run_time) # instantiate the bar

print("Simulating " + str(run_time) + " seconds. Simulation Progress:")
display(f) # display the bar

offset = rospy.Duration(0)
first_run = True
try:
    while delta_t.to_sec() < run_time:
        delta_t = rospy.Time.now() - time_started - offset
        
        if first_run:
            if delta_t.to_sec() < 0:
                offset = delta_t
            first_runt = False
            
        #print delta_t.to_sec(), "(Time offset: ",offset.to_sec(), ")"
        f.value = delta_t.to_sec()
        time.sleep(3)
except KeyboardInterrupt:
    pass

f.bar_style='danger'

sim.pause()
#sim.stop()
walltime_end = time.time()
print("\nSimulation paused at: " + str(delta_t.to_sec()) + ", executed in " + str(walltime_end - walltime_start) + " sec")

INFO: [2018-10-15 12:26:29,324 - Simulation (fable_manipulation_dtu_spiking_cereb_hybrid_0 - localhost #14)] Attempting to transition to state: started
INFO: [2018-10-15 12:26:29,329 - Simulation (fable_manipulation_dtu_spiking_cereb_hybrid_0 - localhost #14)] Simulation state: started

Start delta_t: 0.0 (Should be 0)
Simulating 200 seconds. Simulation Progress:


FloatProgress(value=0.0, max=200.0)

INFO: [2018-10-15 13:30:39,487 - Simulation (fable_manipulation_dtu_spiking_cereb_hybrid_0 - localhost #14)] Attempting to transition to state: paused
INFO: [2018-10-15 13:30:39,563 - Simulation (fable_manipulation_dtu_spiking_cereb_hybrid_0 - localhost #14)] Simulation state: paused

Simulation paused at: 200.052, executed in 3850.23604393 sec


In [71]:
#sim.print_csv_file_names()
sim.save_csv()

###csv_data = {'planner_TF' : sim.get_csv_data('joint_info.csv'), 
###            'controller_TF': sim.get_csv_data('command.csv')}

csv_directory = "/home/dtu-neurorobotics/.opt/nrpStorage/" + experiment +"/"

file_list = os.listdir(csv_directory)
for i in file_list:
    if os.path.isdir(csv_directory+i) and "resources" not in i:
        shutil.copyfile(csv_directory+i+"/"+"joint_info.csv", folder+"/"+"joint_info.csv")
        shutil.copyfile(csv_directory+i+"/"+"command.csv", folder+"/"+"command.csv")

###print "\nLaste entry in CSV at time :", csv_data['planner_TF'][-1][0]
###print ""

#sim.stop()


INFO: [2018-10-15 14:42:50,095 - Simulation (fable_manipulation_dtu_spiking_cereb_hybrid_0 - localhost #14)] Saved CSV data


In [72]:
sim.stop()
vc.delete_cloned_experiment(experiment)

INFO: [2018-10-15 14:42:53,140 - Simulation (fable_manipulation_dtu_spiking_cereb_hybrid_0 - localhost #14)] Attempting to transition to state: stopped
INFO: [2018-10-15 14:42:53,354 - Simulation (fable_manipulation_dtu_spiking_cereb_hybrid_0 - localhost #14)] Simulation has been stopped.
INFO: [2018-10-15 14:42:54,171 - Simulation (fable_manipulation_dtu_spiking_cereb_hybrid_0 - localhost #14)] Simulation state: stopped
INFO: [2018-10-15 14:42:54,172 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-10-15 14:42:54,186 - VirtualCoach] Experiment "fable_manipulation_dtu_spiking_cereb_hybrid_0" deleted successfully


In [73]:
vc.print_cloned_experiments()

INFO: [2018-10-15 14:42:56,219 - VirtualCoach] Retrieving list of experiments.
+------+
| Name |
+======+
+------+
shutdown request: new node registered with same name


In [57]:
sim.stop()

INFO: [2018-10-05 11:55:21,270 - Simulation (fable_manipulation_dtu_spiking_cereb_recurrent_0 - localhost #71)] Attempting to transition to state: stopped
INFO: [2018-10-05 11:55:21,270 - Simulation (fable_manipulation_dtu_spiking_cereb_recurrent_0 - localhost #71)] Attempting to transition to state: stopped


KeyboardInterrupt: 

In [123]:
sim.print_csv_file_names()
sim.save_csv()

###csv_data = {'planner_TF' : sim.get_csv_data('joint_info.csv'), 
###            'controller_TF': sim.get_csv_data('command.csv')}

csv_directory = "/home/dtu-neurorobotics/.opt/nrpStorage/" + experiment +"/"

file_list = os.listdir(csv_directory)
for i in file_list:
    if os.path.isdir(csv_directory+i):
        shutil.copyfile(csv_directory+i+"/"+"joint_info.csv", folder+"/"+"joint_info.csv")
        shutil.copyfile(csv_directory+i+"/"+"command.csv", folder+"/"+"command.csv")

###print "\nLaste entry in CSV at time :", csv_data['planner_TF'][-1][0]
###print ""

sim.stop()


INFO: [2018-09-18 12:41:29,073 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #13)] Attempting to retrieve recorders' csv data
[]
INFO: [2018-09-18 12:41:29,091 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #13)] Saved CSV data


IOError: [Errno 2] No such file or directory: '/home/dtu-neurorobotics/.opt/nrpStorage/fable_manipulation_dtu_spiking_cereb_v2.0_0/csv_records_2018-09-18_12-41-29/joint_info.csv'

In [126]:
vc.delete_cloned_experiment(experiment)

INFO: [2018-09-18 12:44:55,080 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-09-18 12:44:55,094 - VirtualCoach] Experiment "fable_manipulation_dtu_spiking_cereb_v2.0_0" deleted successfully


In [14]:
vc.print_cloned_experiments()

INFO: [2018-10-02 13:29:18,352 - VirtualCoach] Retrieving list of experiments.
+------+
| Name |
+======+
+------+


In [13]:
vc.delete_cloned_experiment("fable_manipulation_dtu_spiking_cereb_recurrent_0")

INFO: [2018-10-02 13:29:15,133 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-10-02 13:29:15,151 - VirtualCoach] Experiment "fable_manipulation_dtu_spiking_cereb_recurrent_0" deleted successfully
